In [ ]:
## skeleton code from : https://github.com/BrianPulfer/PapersReimplementations/blob/master/vit/vit_torch.py

In [ ]:
#!mkdir /content/dataset_1
!unzip /content/raw-img.zip -d /content/dataset_1/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset_1/raw-img/gallina/OIP-HHQnn3Ascn0I4vK_ClazIgDnEs.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hiqwRsp3biM4QXE21c0BKQHaHd.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hj0QkDscMPI-HQO3LDWxCAHaFj.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hj5cCyx7a3mcKtcJboRXAQHaFj.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hjDNyndyKJ4bmKClUttyGwHaHa.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-HJH1KvHRZ5-CtV2QQX4urwHaGU.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-h-JPM5jHqEMVtXX72h-yfwHaJc.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hk8Ca4hOAeS16Cuuk9J4GAHaL-.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-HKEX2YzxomuOiFrtWCbl2gHaGE.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-hkjGSNGzbW2cSxVniEtXFwHaE8.jpeg  
  inflating: /content/dataset_1/raw-img/gallina/OIP-HKvxTyun7wcjjV7tCXCxuQHaE7.jpeg  
  i

In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.image as mpimg
from collections import defaultdict
import dill
import math

import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms

from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GroupKFold
from PIL import Image
import cv2
!pip install pydicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

!pip install einops
from einops import rearrange, repeat

import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 2.0 MB 33.3 MB/s 


In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

In [ ]:
import os 
print(os.getcwd())

/content


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
class MyViT(nn.Module):
    def __init__(self, input_shape, n_patches=7, hidden_d=8, n_heads=2, out_d=10):
        # Super constructor
        super(MyViT, self).__init__()

        # Input and patches sizes
        self.input_shape = input_shape
        self.n_patches = n_patches
        self.n_heads = n_heads
        assert input_shape[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        assert input_shape[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        self.patch_size = (input_shape[1] / n_patches, input_shape[2] / n_patches)
        self.hidden_d = hidden_d

        # 1) Linear mapper
        #print("self input shape ",input_shape)
        self.input_d = int(input_shape[0] * self.patch_size[0] * self.patch_size[1])
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

        # 2) Classification token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))

        # 3) Positional embedding
        # (In forward method)

        # 4a) Layer normalization 1
        self.ln1 = nn.LayerNorm((self.n_patches ** 2 + 1, self.hidden_d))

        # 4b) Multi-head Self Attention (MSA) and classification token
        self.msa = MyMSA(self.hidden_d, n_heads)

        # 5a) Layer normalization 2
        self.ln2 = nn.LayerNorm((self.n_patches ** 2 + 1, self.hidden_d))

        # 5b) Encoder MLP
        self.enc_mlp = nn.Sequential(
            nn.Linear(self.hidden_d, self.hidden_d),
            nn.ReLU()
        )

        # 6) Classification MLP
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_d, out_d),
            nn.Softmax(dim=-1)
        )

    def forward(self, images):
        # Dividing images into patches
        n, c, w, h = images.shape
        #print(n,c,w,h)
        patches = images.reshape(n, self.n_patches ** 2, self.input_d)

        # Running linear layer for tokenization
        tokens = self.linear_mapper(patches)

        # Adding classification token to the tokens
        tokens = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])

        # Adding positional embedding
        tokens += get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d).repeat(n, 1, 1)

        # TRANSFORMER ENCODER BEGINS ###################################
        # NOTICE: MULTIPLE ENCODER BLOCKS CAN BE STACKED TOGETHER ######
        # Running Layer Normalization, MSA and residual connection
        out = tokens + self.msa(self.ln1(tokens))

        # Running Layer Normalization, MLP and residual connection
        out = out + self.enc_mlp(self.ln2(out))
        # TRANSFORMER ENCODER ENDS   ###################################

        # Getting the classification token only
        out = out[:, 0]

        return self.mlp(out)



In [ ]:
class MyMSA(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads

        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(d / n_heads)
        self.q_mappings = [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
        self.k_mappings = [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
        self.v_mappings = [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])


def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result



In [ ]:
from torchvision import datasets, models, transforms

data_transform1 = transforms.Compose([transforms.RandomRotation(45),
                                     transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(1080),
                                     transforms.Resize(512),
                                     transforms.Resize(224),
                                     transforms.RandomRotation(45),
                                      transforms.ToTensor()])

data_transform2 = transforms.Compose([transforms.RandomHorizontalFlip(),
                                     transforms.RandomResizedCrop(1080),
                                     transforms.Resize(224),
                                     transforms.RandomRotation(45),
                                      transforms.RandomRotation(35),
                                      transforms.ToTensor()])
from torch.utils.data import Subset, ConcatDataset , DataLoader
path='/content/dataset_1/raw-img'
dataset1 = datasets.ImageFolder(path,transform=data_transform1)
dataset2 = datasets.ImageFolder(path,transform=data_transform2)
print(type(dataset1))
#master=datasets.ImageFolder(path,transform=data_transform1)
maxlen=750


<class 'torchvision.datasets.folder.ImageFolder'>


In [ ]:
for l, cls in enumerate(dataset1.classes):
    if l == 0 :
        idx = [i for i in range(len(dataset1) ) if dataset1.imgs[i][1] == dataset1.class_to_idx[dataset1.classes[l]]]
        subset = Subset(dataset1, idx)
        master= Subset(subset,idx [:maxlen])
        subset = Subset(dataset2, idx[:maxlen])
        master= ConcatDataset((master, subset))

        print(len(master))
    else :
        idx = [i for i in range(len(dataset1) ) if dataset1.imgs[i][1] == dataset1.class_to_idx[dataset1.classes[l]]]

        subset = Subset(dataset1, idx[:maxlen])
        master= ConcatDataset((master, subset))
        subset = Subset(dataset2, idx[:maxlen])
        master= ConcatDataset((master, subset))
        print(len(master))
        #print(len(master))


1500
3000
4500
6000
7500


In [ ]:
valid_size = 0.1
test_size = 0.1
num_train = len(master)
indices = list(range(num_train))
np.random.shuffle(indices)
valid_split = int(np.floor((valid_size) * num_train))
test_split = int(np.floor((valid_size+test_size) * num_train))
valid_idx, test_idx, train_idx = indices[:valid_split], indices[valid_split:test_split], indices[test_split:]

num_workers = 6
batch_size= 60
disimage = 20
#data = torch.utils.data.DataLoader(master, batch_size=batch_size, num_workers=num_workers)

train_loader = Subset(master, train_idx)
valid_loader = Subset(master,valid_idx )
test_loader = Subset(master,test_idx )



In [ ]:
train_loader =torch.utils.data.DataLoader(train_loader, batch_size=batch_size, num_workers=num_workers)
valid_loader =torch.utils.data.DataLoader(valid_loader, batch_size=batch_size, num_workers=num_workers)
test_loader =torch.utils.data.DataLoader(test_loader, batch_size=batch_size, num_workers=num_workers)


In [ ]:
def main():
    # Loading data
    transform = ToTensor()

    #train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
    #test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

    #train_loader = DataLoader(train_set, shuffle=True, batch_size=16)
    #test_loader = DataLoader(test_set, shuffle=False, batch_size=16)

    # Defining model and training options
    #model = MyViT((3, 224, 224), n_patches=7, hidden_d=20, n_heads=2, out_d=5)
    model = MyViT((3, 224, 224), n_patches=56, hidden_d=50, n_heads=10, out_d=5)
    N_EPOCHS = 20
    LR = 0.01

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()
    for epoch in range(N_EPOCHS):
        train_loss = 0.0
        for batch in train_loader:
            x, y = batch
            y_hat = model(x)
            correct_1=0
            total_1=0
            loss = criterion(y_hat, y) / len(x)

            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            correct_1 += torch.sum(torch.argmax(y_hat, dim=1) == y).item()
            total_1 += len(x)

        print(f"Train accuracy: {correct_1 / total_1 * 100:.2f}%")
        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    correct, total = 0, 0
    test_loss = 0.0
    for batch in test_loader:
        x, y = batch
        y_hat = model(x)
        loss = criterion(y_hat, y) / len(x)
        test_loss += loss

        correct += torch.sum(torch.argmax(y_hat, dim=1) == y).item()
        total += len(x)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")


In [ ]:
if __name__ == '__main__':
    main()

Train accuracy: 13.33%
Epoch 1/20 loss: 2.69
Train accuracy: 13.33%
Epoch 2/20 loss: 2.68
Train accuracy: 11.67%
Epoch 3/20 loss: 2.67
Train accuracy: 15.00%
Epoch 4/20 loss: 2.65
Train accuracy: 25.00%
Epoch 5/20 loss: 2.63


In [ ]:
!rm -rf /content/raw-img/ragno
